## Andrew Byrnes: Fetch Rewards Coding Exercise - Data Analyst
### 2.1-EDA_first_pass.ipynb

This notebook

### Data Sources
- summary : Processed file from notebook 1-Data_Prep

### Changes
- 09-21-2022 : Set up notebook

In [3]:
import pandas as pd
from pathlib import Path
import os
from datetime import datetime
import gzip
import json
import sqlite3

### File Locations

In [4]:
today = datetime.today()
print(today)
db_path = Path.cwd() / "data" / "processed" / "fetch.db"

2022-09-22 01:46:37.808506


### Perform Data Analysis

### Save Excel file into reports directory

Save an Excel file with intermediate results into the report directory

In [ ]:
writer = pd.ExcelWriter(report_file, engine='xlsxwriter')

In [ ]:
df.to_excel(writer, sheet_name='Report')

In [ ]:
writer.save()